### Building Langgraph agents with human in the loop
In this notebook, we will be experimenting with builing langgraph agent exectors while supporting human feedback for task execution

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain import hub
from langchain_core.messages import AIMessage, FunctionMessage, BaseMessage, HumanMessage
from langchain_core.agents import AgentFinish, AgentAction
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.tools.tavily_search.tool import TavilySearchResults

In [4]:
tools = [TavilySearchResults(max_results=1), PythonREPLTool()]
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")
agent_runnable = create_openai_functions_agent(llm, tools=tools, prompt=prompt)

In [11]:
# defining the agent state
from typing import Annotated, Sequence, Tuple, TypedDict, Union
import operator


class AgentState(TypedDict):
    input : Annotated[str, operator.setitem]
    
    chat_history : Annotated[Sequence[BaseMessage], operator.add] = []
    
    agent_outcome : Annotated[Union[AgentAction, AgentFinish, None], operator.setitem] = None
    
    intermediate_steps: Annotated[Sequence[Tuple[AgentAction, str]], operator.add] = []

In [12]:
# define the noes that would be used in our graph
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor


tool_executor = ToolExecutor(tools)

def run_agent(state: AgentState):
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


def execute_tools(state: AgentState):
    agent_action = state["agent_outcome"]
    response = input(prompt=f"[y/n] continue with {agent_action}?")
    if response == "n":
        raise ValueError("Action not approved by user")
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}
    
    
def should_continue(state: AgentState):
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

#### Define the graph
Now that we have all the individual nodes defined, we can now define the graph and compile it into an application

In [13]:
from langgraph.graph import StateGraph, END

In [14]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.set_entry_point("agent")


workflow.add_conditional_edges("agent", should_continue, {
    "continue": "action", 
    "end": END
})

workflow.add_edge("action", "agent")


app = workflow.compile()

In [16]:
inputs = {"input": "what is the weather in sf", "chat_history": [], "intermediate_steps": [],}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentFinish(return_values={'output': ''}, log='')}
----
{'input': 'what is the weather in sf', 'chat_history': [], 'agent_outcome': AgentFinish(return_values={'output': ''}, log=''), 'intermediate_steps': []}
----
